In [32]:
# Get the clicked coordinates
# can use this to pick the top left corner and bottom right corner. we can then use this to
# extract all the squares from the sudoku, as in the notebook sudoku image read
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
#This will display all the available mouse click events  
#events = [i for i in dir(cv2) if 'EVENT' in i]
#print(events)

#click event function
def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        refPt.append([x,y])
        #font = cv2.FONT_HERSHEY_SIMPLEX
        #strXY = str(x)+", "+str(y)
        #cv2.putText(img, strXY, (x,y), font, 0.5, (255,255,0), 2)
        cv2.circle(img,(x,y),2,(255, 0, 0),2)
        cv2.imshow("image", img)

    '''
    if event == cv2.EVENT_RBUTTONDOWN:
        blue = img[y, x, 0]
        green = img[y, x, 1]
        red = img[y, x, 2]
        font = cv2.FONT_HERSHEY_SIMPLEX
        strBGR = str(blue)+", "+str(green)+","+str(red)
        cv2.putText(img, strBGR, (x,y), font, 0.5, (0,255,255), 2)
        cv2.imshow("image", img)
    '''



def ensureSquare(listIs):
    lensW = listIs[1][0]-listIs[0][0]
    lensH = listIs[2][1]-listIs[1][1]
    diffL = abs(lensW-lensH)
    if lensW>lensH:
        listIs[1][0] = listIs[1][0]-diffL
        listIs[3][0] = listIs[3][0]-diffL
    if lensH>lensW:
        listIs[2][1] = listIs[2][1]-diffL
        listIs[3][1] = listIs[3][1]-diffL
    return(listIs)

def getDst(start):
    dst = [[min(start[0][0], start[2][0]), min(start[0][1], start[1][1])],
           [max(start[1][0], start[3][0]), min(start[0][1], start[1][1])],
          [min(start[0][0], start[2][0]), max(start[2][1], start[3][1])],
          [max(start[1][0], start[3][0]), max(start[2][1], start[3][1])]]
    return(dst)

def formatForWrap(listIs):
    res = np.float32([(listIs[0][0],listIs[0][1]),
                     (listIs[1][0],listIs[1][1]),
                     (listIs[2][0],listIs[2][1]),
                     (listIs[3][0],listIs[3][1])])
    return(res)
def unwarp(img, src, dst):
    h, w = img.shape[:2]
    # use cv2.getPerspectiveTransform() to get M, the transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # use cv2.warpPerspective() to warp your image to a top-down view
    warped = cv2.warpPerspective(img, M, (w, h), flags=cv2.INTER_LINEAR)
    return(warped)

def drawGrid(img, coords):
    coords = dstList.copy()
    for i in range(0,len(coords)):
        for ii in range(0,2):
            coords[i][ii] = int(coords[i][ii])
        
    lens = coords[1][0]-coords[0][0]
    cellWidth= int(lens/9)
    colourIs = (0,0,255)

    for i in range(0,10):
        cv2.line(img,(coords[0][0],
                      coords[0][1]+(i*cellWidth)),
                 (coords[0][0]+(9*cellWidth),
                  coords[1][1]+(i*cellWidth)),
                 colourIs,2)
        cv2.line(img,(coords[0][0]+(i*cellWidth),
                      coords[0][1]),
                 (coords[0][0]+(i*cellWidth),
                  coords[1][1]+(9*cellWidth)),
                 colourIs,2)
    plt.imshow(img)
    plt.show()

def saveImg(img, coords, file):
    for i in range(0,len(coords)):
        for ii in range(0,2):
            coords[i][ii] = int(coords[i][ii])
        
    lens = coords[1][0]-coords[0][0]
    cellWidth= int(lens/9)
    colourIs = (0,0,255)

    imCounter = 0
    for i in range(0,9):
        for ii in range(0,9):
            im = img[coords[0][1]+(i*cellWidth):coords[0][1]+((i+1)*cellWidth),
                    coords[0][0]+(ii*cellWidth):coords[0][0]+((ii +1)*cellWidth),:]
            cv2.imwrite('./allForTraining/{}_{}.png'.format(file, imCounter),im)
            imCounter = imCounter + 1

def rescaleImg(img):
    
    scale_fraction = [1] # fraction of original size

    if img.shape[0]>480:# good for visualization on my laptop
        scale_fraction.append(480/img.shape[0])
    if img.shape[0]>640:# good for visualization on my laptop
        scale_fraction.append(640/img.shape[1])
    scale_fraction = min(scale_fraction)
    width = int(img.shape[1] * scale_fraction)
    height = int(img.shape[0] * scale_fraction)
    dim = (width, height)
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return(img)

In [33]:
dataDir = 'D:\\repos\\sudokuSolver_py\\rawImg\\'
allFiles = os.listdir(dataDir)

#filesDone = []
for file in allFiles:
    if file.endswith(".jpg"):
        if not file in filesDone:
            #This variable we use to store the pixel location
            refPt = []

            img = cv2.imread(dataDir+file)
            img = rescaleImg(img)

            cv2.imshow("image", img)

            #calling the mouse click event
            cv2.setMouseCallback("image", click_event)

            cv2.waitKey(0)
            cv2.destroyAllWindows()

            src = formatForWrap(refPt)
            dstList = getDst(src)
            dstList = ensureSquare(dstList)
            dst = formatForWrap(dstList)
            warped = unwarp(img, src, dst)
            saveImg(warped, dstList, file)
            filesDone.append(file)
print('Done')